# 練習録のデータの移行(参加者データを除く)
### postgresqlへの接続

In [ ]:
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.errors import DuplicateColumn

conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

def get_data(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
    return data

def get_data_dict(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
        col_name = [col.name for col in cur.description]
    return [ dict(zip(col_name,d)) for d in data]

def get_colname(table):
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM {} LIMIT 1;'.format(table))
        data = [col.name for col in cur.description]
    return data

def exec_transaction(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
    conn.commit()

### postgresql上での練習録id情報の変換(str->int)

In [ ]:
# add training_id
exec_transaction('''CREATE SEQUENCE IF NOT EXISTS training_id_seq ;''')
try:
    exec_transaction('''ALTER TABLE training ADD COLUMN training_id integer NOT NULL DEFAULT nextval('training_id_seq');''')
except DuplicateColumn as err:
    print(err)
    conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

### mysqlへの接続

In [ ]:
import mysql.connector
conn2 = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'honomara',
    password =  'honomara',
    database = 'honomara',
)

cur2 = conn2.cursor(dictionary=True)

### trainingテーブルの作成

In [ ]:
create_training_table ="""
CREATE TABLE IF NOT EXISTS training (
    `id`      INT           PRIMARY KEY AUTO_INCREMENT,
    `date`    DATE          NOT NULL,
    `place`   VARCHAR(30)   NOT NULL,
    `weather` VARCHAR(30),
    `title`   VARCHAR(100)  NOT NULL,
    `comment` TEXT,
    INDEX USING BTREE (date)
);
"""

cur2.execute(create_training_table)
conn2.commit()

In [ ]:
def get_data(row):
    data = (row['training_id'],str(row['date']),row['site'],row['weather'],row['subject'],row['comment'])
    return data

query = "INSERT INTO training (id,date,place,weather,title,comment) VALUES (%s,%s,%s,%s,%s,%s);"
for row in get_data_dict('SELECT * FROM training;'):
    cur2.execute(query,get_data(row))
conn2.commit()

## miscellaneous info

In [ ]:
get_colname('training')

In [ ]:
get_data('SELECT wday,count(*) FROM training GROUP BY wday;')

In [ ]:
get_data("SELECT site,count(*) AS cnt FROM training GROUP BY site ORDER BY cnt DESC;")

In [ ]:
cur2.execute('SHOW COLUMNS FROM training;')
for d in cur2.fetchall():
    print(d['Field'],d['Type'])